# Assignment 3

Import libraries and define common helper functions

In [1]:
import os
import sys
import gzip
import json
from pathlib import Path
import csv

import pandas as pd
import s3fs
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
import fastavro
import pygeohash
import snappy
import jsonschema
from jsonschema.exceptions import ValidationError


endpoint_url='https://storage.budsc.midwest-datascience.com'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)


def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        

    return records

Load the records from https://storage.budsc.midwest-datascience.com/data/processed/openflights/routes.jsonl.gz 

In [2]:
records = read_jsonl_data()

## 3.1

### 3.1.a JSON Schema

In [3]:
def validate_jsonl_data(records):
    schema_path = schema_dir.joinpath('routes-schema.json')
    with open(schema_path) as f:
        schema = json.load(f)
    validation_csv_path = results_dir.joinpath('validation-results.csv')
    # Open the validation CSV
    with open(validation_csv_path, 'w', encoding="utf-8") as f:
        # Create column names
        fieldnames = ['row', 'valid', 'msg']
        # Assign CSV writer object
        csv_writer = csv.DictWriter(f, fieldnames=fieldnames, lineterminator = '\n')
        csv_writer.writeheader()   
        for i, record in enumerate(records):
            try:
                ## TODO: Validate record 
                result = dict(
                    row = i,
                    valid = True,
                    msg = record
                )
                pass
            except ValidationError as e:
                ## Print message if invalid record
                result = dict(
                    row = i,
                    valid = False,
                    msg = record
                )
                pass
            finally:
                # Write the line to the CSV.
                csv_writer.writerow(result)

validate_jsonl_data(records)

### 3.1.b Avro

In [4]:
def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')
    ## TODO: Use fastavro to create Avro dataset
    with open(schema_path) as f:
        schema = json.load(f)  
    with open(data_path, 'wb') as out:
        fastavro.writer(out, schema, records)
        
create_avro_dataset(records)

In [7]:
current_dir

PosixPath('/home/jovyan/dsc650/dsc650/assignments/assignment03')

### 3.1.c Parquet

In [8]:
def create_parquet_dataset():
    src_data_path = '/home/jovyan/dsc650/data/processed/openflights/routes.jsonl.gz'
    parquet_output_path = results_dir.joinpath('routes.parquet')
    # Opeining routes.json
    with open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            # reading each table from json
            table = pa.json.read_json(f)
            # Simply writing to the json
            pq.write_table(table, parquet_output_path)   

create_parquet_dataset()

### 3.1.d Protocol Buffers

In [9]:
sys.path.insert(0, os.path.abspath('routes_pb2'))

import routes_pb2

def _airport_to_proto_obj(airport):
    obj = routes_pb2.Airport()
    if airport is None:
        return None
    if airport.get('airport_id') is None:
        return None

    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj


def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    ## TODO: Create an Airline obj using Protocol Buffers API
    if not airline.get('name'):
        return None
    if not airline.get('airline_id'):
        return None
    obj.airline_id = airline.get('airline_id')
    obj.name = airline.get('name')
    if airline.get('alias'):
        obj.alias = airline.get('alias')
    ## TODO
    if airline.get('iata'):
        obj.iata = airline.get('iata')
    # icao
    if airline.get('icao'):
        obj.icao = airline.get('icao')
    # callsign
    if airline.get('callsign'):
        obj.callsign = airline.get('callsign')
    # country
    if airline.get('country'):
        obj.country = airline.get('country')
    
    obj.active = airline.get('active')
    return obj


def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()
        
        # Source airport
        src_airport = _airport_to_proto_obj(record.get('src_airport'))
        # Destination airport
        dst_airport = _airport_to_proto_obj(record.get('dst_airport'))
        # Airline
        airline = _airline_to_proto_obj(record.get('airline'))
        # Code Share
        codeshare = record.get('codeshare')
        # Equipment
        equipment = record.get('equipment')
        
        # Some values need to be occupied in the proto-buffer
        # so if they are empty, impute them with 0s
        if not src_airport == None:
            route.src_airport.CopyFrom(src_airport)
        else:
            # Mandatory values
            route.src_airport.airport_id = 0
            route.src_airport.latitude = 0
            route.src_airport.longitude = 0
            
        if not dst_airport == None:
            route.dst_airport.CopyFrom(dst_airport)
        else:
            # Mandatory values
            route.dst_airport.airport_id = 0
            route.dst_airport.latitude = 0
            route.dst_airport.longitude = 0
            
        if not airline == None:
            route.airline.CopyFrom(airline)
        else:
            # Mandatory values
            route.airline.airline_id = 0
            route.airline.name = None
            route.airline.active = False

        route.codeshare = codeshare
        route.equipment.extend(equipment)
        
        routes.route.append(route)

    # Uncompressed
    data_path = results_dir.joinpath('routes.pb')
    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    # Snappy Compression
    compressed_path = results_dir.joinpath('routes.pb.snappy')
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
        
create_protobuf_dataset(records)

### 3.1.e Output

In [10]:
comparison_csv_path = results_dir.joinpath('comparison.csv')

# Open CSV
with open(comparison_csv_path, 'w', encoding="utf-8") as f:
    # Column Names
    fieldnames = ['file', 'size_MB']
    
    # CSV writer 
    csv_writer = csv.DictWriter(f, fieldnames=fieldnames, lineterminator = '\n')
    csv_writer.writeheader()
    
    # Iterate over the files in the results directory
    for files in os.walk(results_dir):
        try:
            # Check to make sure file isn't ran more than once
            for file in files[2]:
                if file == "comparison.csv":
                    next
                else:
                    print(f"{file:>25}: {int(os.stat(results_dir.joinpath(file))[6]) / 1024 / 1024:.2f}MB")
                    file_size = dict(file = file, size_MB = int(os.stat(results_dir.joinpath(file))[6]) / 1024 / 1024)
                    csv_writer.writerow(file_size)
                
        # Catch for running geohash to many times
        except FileNotFoundError:
            pass

   validation-results.csv: 57.08MB
              routes.avro: 18.74MB
           routes.parquet: 1.88MB
         routes.pb.snappy: 3.59MB
                routes.pb: 21.50MB


## 3.2

### 3.2.a Simple Geohash Index

In [11]:
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    for record in records:
        src_airport = record.get('src_airport', {})
        if src_airport:
            latitude = src_airport.get('latitude')
            longitude = src_airport.get('longitude')
            if latitude and longitude:
                ## TODO: use pygeohash.encode() to assign geohashes to the records and complete the hashes list
                hashes.append(pygeohash.encode(longitude=longitude,latitude=latitude,precision=3))
                record['geohash'] = pygeohash.encode(longitude=longitude,latitude=latitude,precision=3)
    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))
    hash_index = {value: [] for value in three_letter}
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))
            
create_hash_dirs(records)

### 3.2.b Simple Search Feature

In [18]:
def airport_search(latitude, longitude):
    src_geohash_code = pygeohash.encode(longitude=longitude,latitude=latitude,precision=15)
    # load the data from that file
    geoindex_dir = results_dir.joinpath('geoindex')
    # building my search area
    my_geo_dir = geoindex_dir.joinpath(str(src_geohash_code[0]))
    my_geo_dir = my_geo_dir.joinpath(str(src_geohash_code[:2]))
    my_geo_dir = my_geo_dir.joinpath(str(src_geohash_code[:3])+".jsonl.gz")

    # print(my_geo_dir)
    with open(my_geo_dir, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            geo_records = [json.loads(line) for line in f.readlines()]
            current_dis = 0
            closest_airport = {}
            for distance in geo_records:
                if distance['src_airport']:
                    temp_lat = distance["src_airport"]["latitude"]
                    temp_lon = distance["src_airport"]["longitude"]
                    dst_geohash_code = pygeohash.encode(longitude=temp_lon,latitude=temp_lat,precision=15)
                    dis = pygeohash.geohash_approximate_distance(str(src_geohash_code), str(dst_geohash_code))
                    if current_dis == 0 or dis <= current_dis:
                        current_dis = dis
                        closest_airport = distance["airline"]["name"]

            print("The closest airport is:" + closest_airport + ", which is: "+ str(current_dis) + " km away")

airport_search(41.1499988, -95.91779)

The closest airport is:Southwest Airlines, which is: 19545 km away
